# H&M Personalized Fashion Recommendations
- 日本語コメント付きのベースラインモデルです
- 以下のnotebookをリファクタしたものになります
    - 元notebook
        - H&M Pure Pytorch Baseline
        - https://www.kaggle.com/code/aerdem4/h-m-pure-pytorch-baseline
    - リファクタ内容
        - DEBUGモード追加
        - コメント付与
        - 定数のみ切り出し

In [ ]:
import gc
import numpy as np
import pandas as pd
import torch

In [ ]:
# DEBUG = True
DEBUG = False
# ORIGIN = True
ORIGIN = False

# ファイルパス
# baselineではTRANSACTIONのみ学習に使用している
TRANSACTION_PATH = "../input/h-and-m-personalized-fashion-recommendations/transactions_train.csv"
ARTICLES_PATH = "../input/h-and-m-personalized-fashion-recommendations/articles.csv"
CUSTOMERS_PATH = "../input/h-and-m-personalized-fashion-recommendations/customers.csv"
SAMPLE_SUB_PATH = '../input/h-and-m-personalized-fashion-recommendations/sample_submission.csv'
IMAGES_DIR = "../input/h-and-m-personalized-fashion-recommendations/images/"

# 前処理
ACTIVE_DATE_TH = "2019-09-01" # この日付より最近購入された商品のみ学習/推論対象とする
WEEK_HIST_MAX = 5 # 過去何日分の購買履歴を特徴量とするか
VAL_WEEKS = [0] # どの週のデータをvalidationデータとするか
TRAIN_WEEKS = [1, 2, 3, 4] # どの週のデータをtrainデータとするか

# データセット
SEQ_LEN = 16 # 何個分の購買履歴まで使うか
BS = 256 # バッチサイズ
NW = 8 # ワーカーズ

# 学習
DIM_ARTICLE = 512 # article_idを何次元の特徴量にembeddingするか
INIT_LR = 3e-4
BETAS = (0.9, 0.999)
EPS=1e-08
LR_DICT = { # n-epoch目の時、n < KEYならVALをlrとする
    1: 5e-5,
    6: 1e-3, 
    9: 1e-4,
    100: 1e-5
}
MODEL_NAME = "exp001"
SEED = 0
BASE_EPOCHS = 5 # ベースモデルのエポック数
FT_EPOCHS = 5 # fine tuningのエポック数
FT_WEEKS = 4 # fine tuningする際使用するweek数

In [ ]:
# データ読み込み
df = pd.read_csv(TRANSACTION_PATH, dtype={"article_id": str})
print(df.shape)
df.head()

if DEBUG:
    print("DEBUGモードで処理を実行します")
    BASE_EPOCHS = 1
    FT_EPOCHS = 1
    SEQ_LEN = 5
    DIM_ARTICLE = 8
    df = df.sample(frac=0.005)
    print(df.shape)
    display(df.head())

In [ ]:
# datetime型に変換
df["t_dat"] = pd.to_datetime(df["t_dat"])
df["t_dat"].max()

# 前処理(1)

In [ ]:
# 直近で購入された商品一覧を抽出する
active_articles = df.groupby("article_id")["t_dat"].max().reset_index()
active_articles = active_articles[active_articles["t_dat"] >= ACTIVE_DATE_TH].reset_index()
active_articles.shape

In [ ]:
# 購買履歴のうち直近で購入された商品IDに対応する履歴のみを抽出
df = df[df["article_id"].isin(active_articles["article_id"])].reset_index(drop=True)
df.shape

In [ ]:
# "week"カラムを追加(weekが小さいほど最近)
df["week"] = (df["t_dat"].max() - df["t_dat"]).dt.days // 7
df["week"].value_counts()

if DEBUG:
    # week=0がvalidationデータになる
    # validationデータはtransactionの最後の一週間(2020-09-16以降)とする
    display(df[df["week"] == 0].t_dat.describe())

In [ ]:
# articleIdをラベルに変換
from sklearn.preprocessing import LabelEncoder
article_ids = np.concatenate([["placeholder"], np.unique(df["article_id"].values)])
le_article = LabelEncoder()
le_article.fit(article_ids)
df["article_id"] = le_article.transform(df["article_id"])

In [ ]:
if DEBUG:
    print("データ(head)")
    display(df.head())
    print("データ(describe)")
    display(df.describe())
    print("データ(info)")
    display(df.info())

# articleの前処理

In [ ]:
article_df = pd.read_csv(ARTICLES_PATH, dtype={"article_id": str})
print(article_df.shape)
article_df.head()

In [ ]:
# 分析対象のarticleに絞る
article_df = article_df[article_df["article_id"].isin(article_ids)]
print(article_df.shape)

In [ ]:
# transactionsと同じくラベルエンコーディング
article_df["article_id"] = le_article.transform(article_df["article_id"])
print(article_df.shape)
article_df.head()

In [ ]:
# 全てのカラムをラベルエンコーディング
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
article_df = article_df.apply(le.fit_transform)


In [ ]:
# 正規化
article_df=(article_df-article_df.min())/(article_df.max()-article_df.min())

In [ ]:
def articleid2vector(article_id: int)->torch.Tensor:
    """
    任意のarticleidをembeddingする
    input: ラベルエンコード後のarticle_id
    output: embedding結果
        <class 'torch.Tensor'>
        torch.Size([任意の特徴量次元])
    """
    article_vec = article_df[article_df["article_id"] == article_id]
    article_vec = article_vec.drop("article_id", axis=1).iloc[0].to_list()
    x = np.array(article_vec)
    return x

if ORIGIN:
    pass
else:
    DIM_ARTICLE = len(articleid2vector(0))

# customerの前処理

In [ ]:
# customer_df = pd.read_csv(ARTICLES_PATH, dtype={"article_id": str})
# print(article_df.shape)
# article_df.head()

In [ ]:
# df = pd.merge(df, article_df, how="left")

# 前処理(2)

In [ ]:
def create_dataset(df, week, debug=False) -> pd.DataFrame:
    """
    データセットを作成
    各顧客が指定した週で購入したものと、過去n週で購入したもの
    
    【データの説明】
        customer_id: 顧客ID
        target: (その週に)何を購入したか←予測したいもの
        week: 週
        article_id: (過去n週で)何を購入したか
        week_history: (過去n週の)どの週で購入したか
        
    【処理の流れ】
    1.指定した週の過去n週間分のデータを抽出する
    2.顧客(customer_id)毎に買った商品リスト(article_idリスト)、購入したweekリストを作成
    3.購入したweekリストをweek_historyとする
    4.指定した週のレコードをtarget_dfとして抽出
    5.target_dfも顧客(customer_id)毎に買った商品リスト(article_idリスト)を作成
    6.買った商品リストのカラムをtargetに変換
    7.target_dfのweekカラムに指定した週の値を代入
    8.customer_idでleft join
    """
    hist_df = df[(df["week"] > week) & (df["week"] <= week + WEEK_HIST_MAX)]
    hist_df = hist_df.groupby("customer_id").agg({"article_id": list, "week": list}).reset_index()
    hist_df.rename(columns={"week": 'week_history'}, inplace=True)
    if debug:
        display(hist_df.head())
    
    target_df = df[df["week"] == week]
    target_df = target_df.groupby("customer_id").agg({"article_id": list}).reset_index()
    target_df.rename(columns={"article_id": "target"}, inplace=True)
    target_df["week"] = week
    result_df = target_df.merge(hist_df, on="customer_id", how="left")
    if debug:
        display(result_df.head())
    return result_df

In [ ]:
# データセット作成
val_df = pd.concat([create_dataset(df, w) for w in VAL_WEEKS]).reset_index(drop=True)
train_df = pd.concat([create_dataset(df, w) for w in TRAIN_WEEKS]).reset_index(drop=True)
train_df.shape, val_df.shape

del df
print(gc.collect())

if DEBUG:
    print("validationデータ(head)")
    display(val_df.head())
    print("trainデータ(head + tail)")
    display(train_df.head())
    display(train_df.tail())

# モデル学習

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch
from tqdm import tqdm

class HMDataset(Dataset):
    """
    pytorch Datasetを継承する際には以下の2つの関数をoverrideする
    ①.def __getitem__(self, index: int):
        indexを与えられたときに対応するデータを返す  
    ②.def __len__(self):
        データのサンプル数を返す
    """
    def __init__(self, df, seq_len, is_test=False):
        """
        df: 前処理したデータセット
        seq_len: 特徴量の次元数
        is_test: テストデータか否か
        """
        self.df = df.reset_index(drop=True)
        self.seq_len = seq_len
        self.is_test = is_test
    
    def __len__(self):
        return self.df.shape[0]
    
    def __getitem__(self, index):
        """
        対象indexデータを加工して返す
        
        特徴量：
        article_hist: 過去n週で購買した商品(要素数=seq_len)
        week_hist: article_histの商品をいつ購入したかを正規化(0~1)した値(要素数=seq_len)
        目的変数: 
        target: この週どの商品を購入したか(要素数=len(article_ids))
        """
        
        # dfのindex番目の行を抽出
        row = self.df.iloc[index]
        
        if self.is_test:
            # testならtargetは長さ2のlist
            target = torch.zeros(2).float()
        else:
            # trainならtargetは長さが"対象とする商品数"のlist
            target = torch.zeros(len(article_ids)).float()
            
            # targetのうち顧客が購入したもの(article_idのindex)を1.0にする
            for t in row.target:
                target[t] = 1.0
            
        # 購入履歴は長さseq_lenのlist
        article_hist = torch.zeros(self.seq_len).long()
        week_hist = torch.ones(self.seq_len).float()
        
        # 何かしらの購入履歴があるか
        if isinstance(row.article_id, list):
            # 購入履歴の数がseq_lenより長いなら直近の履歴から抽出
            if len(row.article_id) >= self.seq_len:
                article_hist = torch.LongTensor(row.article_id[-self.seq_len:])
                week_hist = (torch.LongTensor(row.week_history[-self.seq_len:]) - row.week)/WEEK_HIST_MAX/2
            # 購入履歴の数がseq_lenより短いならarticle_hist/week_histの前半indexに購入履歴を格納
            else:   
                article_hist[-len(row.article_id):] = torch.LongTensor(row.article_id)
                week_hist[-len(row.article_id):] = (torch.LongTensor(row.week_history) - row.week)/WEEK_HIST_MAX/2
        
        # 八木追加: ==========
        article_vecs = []
        for article_id in article_hist:
            article_vec = articleid2vector(int(article_id))
            article_vecs.append(article_vec)
        article_hist_vec = torch.from_numpy(np.array(article_vecs).astype(np.float32)).clone()
        
        if ORIGIN:
            return article_hist, week_hist, target
        else:
            return article_hist_vec, week_hist, target
#         
    
if DEBUG:
    print("train_dfでHMDataSetを作成し中身を確認")
    print("[0]article_hist: 過去n週で購入した商品一覧")
    print("[1]week_hist: 各商品を過去n週のうちどの週で購入したか一覧")
    print("[2]target: 今週どの商品を購入したか0:購入しなかった、1:購入した(要素数: len(article_ids)")
    print("-"*60)
    print()
    print("データセットの中身")
    sample = HMDataset(train_df, 64)[0]
    display(sample)
    print()
    print("各特徴量のlen")
    print(len(sample[0]))
    print(len(sample[1]))
    print(len(sample[2]))

In [ ]:
def adjust_lr(optimizer, epoch):
    """
    エポックによって特徴量を変える
    """
    
    for epoch_th in LR_DICT.keys():
        if epoch < epoch_th:
            lr = LR_DICT[epoch_th]

    for p in optimizer.param_groups:
        p['lr'] = lr
    return lr
    
def get_optimizer(net):
    """
    最適化関数を取得
    """
    optimizer = torch.optim.Adam(
        params=filter(lambda p: p.requires_grad, net.parameters()),
        lr=INIT_LR,
        betas=BETAS,
        eps=EPS
    )
    return optimizer

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

class HMModel(nn.Module):
    """
    ネットワークを定義
    一方通行ではない複雑なモデル（ネットワーク）を構築するには、torch.nn.Moduleを継承したサブクラスを定義する
    
    基本的な書き方
    __init__()
    　- 使用するモジュール（レイヤー）のインスタンスを生成
     ※super().__init__()を忘れないように注意
    forward()
    　- レイヤーを所望の順番で適用していく
    """
    def __init__(self, article_shape):
        """
        article_shape = (商品の種類数, ベクトルの次元)
        """
        # おまじない
        super(HMModel, self).__init__()
        
        # レイヤー(Sequentialはレイヤーのひとまとまり?)
        # nn.Embedding: IDに対応する初期ベクトルを作成(IDをベクトルに埋め込む)
        # nn.Parameter: レイヤーのパラメータ
        self.article_emb = nn.Embedding(num_embeddings=article_shape[0], embedding_dim=article_shape[1])
        self.article_likelihood = nn.Parameter(torch.zeros(article_shape[0]), requires_grad=True)
        self.top = nn.Sequential(
            nn.Conv1d(3, 32, kernel_size=1),
            nn.LeakyReLU(),
            nn.Conv1d(32, 8, kernel_size=1),
            nn.LeakyReLU(),
            nn.Conv1d(8, 1, kernel_size=1)
        )
        
    def forward(self, inputs):
        """
        inputs: 特徴量
        """

        # 商品IDをランダムなベクトルに変換
        # →ここの埋め込み方をよりいいものにすればいいのでは??
        if ORIGIN:
            article_hist, week_hist = inputs[0], inputs[1]
            x = self.article_emb(article_hist)
        else:
            article_hist_vec, week_hist = inputs[0], inputs[1]
            x = article_hist_vec
        
        x = F.normalize(x, dim=2)
        x = x@F.normalize(self.article_emb.weight).T

        x, indices = x.max(axis=1)
        x = x.clamp(1e-3, 0.999)
        x = -torch.log(1/x - 1)
        
        max_week = week_hist.unsqueeze(2).repeat(1, 1, x.shape[-1]).gather(1, indices.unsqueeze(1).repeat(1, week_hist.shape[1], 1))
        max_week = max_week.mean(axis=1).unsqueeze(1)
        
        x = torch.cat([
                x.unsqueeze(1),
                max_week,
                self.article_likelihood[None, None, :].repeat(x.shape[0], 1, 1)
                ], axis=1)
        
        # ネットワークで計算
        x = self.top(x).squeeze(1)
        return x
    
    
model = HMModel((len(le_article.classes_), DIM_ARTICLE))
model = model.cuda()

if DEBUG:
    print("モデル構造を可視化")
    print(model)


In [ ]:
import sys

def calc_map(topk_preds, target_array, k=12):
    """
    MAPスコアを計算
    topk_perds: 予測結果
    target_array: 正解
    k: 予測件数(今回のコンペだと上位12件を予測)
    """
    metric = []
    tp, fp = 0, 0
    
    # 予測結果を1件ずつ見ていきtp/fpを計算
    for pred in topk_preds:
        if target_array[pred]:
            tp += 1
            metric.append(tp/(tp + fp))
        else:
            fp += 1
            
    return np.sum(metric) / min(k, target_array.sum())

def read_data(data):
    """
    TODO: おまじない的なやつ?
    """
    return tuple(d.cuda() for d in data[:-1]), data[-1].cuda()


def validate(model, val_loader, k=12):
    """
    バリデーション
    """
    model.eval()
    
    tbar = tqdm(val_loader, file=sys.stdout)
    
    maps = []
    
    with torch.no_grad():
        for idx, data in enumerate(tbar):
            # inputs(入力特徴量),target(正解)を抽出
            inputs, target = read_data(data)
            
            # モデルで予測
            logits = model(inputs)
            
            # topk(スコア上位k件を抽出)
            _, indices = torch.topk(logits, k, dim=1)

            # GPU→CPUへ
            indices = indices.detach().cpu().numpy() # 予測結果
            target = target.detach().cpu().numpy() # 正解

            # スコア計算
            for i in range(indices.shape[0]):
                maps.append(calc_map(indices[i], target[i]))
        
    return np.mean(maps)

# DataSet: 学習に使うデータ(前処理などを定義している)
val_dataset = HMDataset(val_df, SEQ_LEN)

# DataLoader: DataSetをバッチで処理するためのもの
val_loader = DataLoader(
    val_dataset,
    batch_size=BS,
    shuffle=False,
    num_workers=NW,
    pin_memory=False,
    drop_last=False
)

### Train and validate

In [ ]:
def dice_loss(y_pred, y_true):
    """
    dice_loss
    Dice損失は2つの要素の類似度の評価するために使われているDice係数(F値)を損失として用いたもの
    """
    y_pred = y_pred.sigmoid()
    intersect = (y_true*y_pred).sum(axis=1)
    
    return 1 - (intersect/(intersect + y_true.sum(axis=1) + y_pred.sum(axis=1))).mean()


def train(model, train_loader, val_loader, epochs):
    np.random.seed(SEED)
    
    # 最適化関数を定義
    optimizer = get_optimizer(model)
    scaler = torch.cuda.amp.GradScaler() # ←高速化のための何かっぽい

    # ロスを定義
    criterion = torch.nn.BCEWithLogitsLoss()
    
    for e in range(epochs):
        model.train()
        tbar = tqdm(train_loader, file=sys.stdout)
        
        lr = adjust_lr(optimizer, e)
        
        loss_list = []

        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            optimizer.zero_grad()
            
            with torch.cuda.amp.autocast():
                logits = model(inputs)
                
                # ロスは2種類を組み合わせている(完全一致と類似度？)
                loss = criterion(logits, target) + dice_loss(logits, target)
            
            #loss.backward()
            scaler.scale(loss).backward()
            #optimizer.step()
            scaler.step(optimizer)
            scaler.update()
            
            loss_list.append(loss.detach().cpu().item())
            
            avg_loss = np.round(100*np.mean(loss_list), 4)

            tbar.set_description(f"Epoch {e+1} Loss: {avg_loss} lr: {lr}")
            
        val_map = validate(model, val_loader)

        log_text = f"Epoch {e+1}\nTrain Loss: {avg_loss}\nValidation MAP: {val_map}\n"
            
        print(log_text)
        
        logfile = open(f"{MODEL_NAME}_{SEED}.txt", 'a')
        logfile.write(log_text)
        logfile.close()
    return model

train_dataset = HMDataset(train_df, SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=NW,
                          pin_memory=False, drop_last=True)

model = train(model, train_loader, val_loader, epochs=BASE_EPOCHS)

### Finetune with more recent data for submission (include validation set)
今回のコンペは購買履歴の次の週の結果を予測する  
→ 直近のデータの方が重要と考えられる  
→ 最後にバリデーションデータを使ってfine tuningし直す

In [ ]:
train_dataset = HMDataset(train_df[train_df["week"] < FT_WEEKS].append(val_df), SEQ_LEN)
train_loader = DataLoader(train_dataset, batch_size=BS, shuffle=True, num_workers=NW,
                          pin_memory=False, drop_last=True)

model = train(model, train_loader, val_loader, epochs=FT_EPOCHS)

In [ ]:
test_df = pd.read_csv(SAMPLE_SUB_PATH).drop("prediction", axis=1)
print(test_df.shape)
test_df.head()

In [ ]:
def create_test_dataset(test_df):
    """
    テストデータ(submitする際の予測データ)を作成する
    """
    week = -1
    test_df["week"] = week
    
    # 過去n週間で買ったものを列として追加していく
    hist_df = df[(df["week"] > week) & (df["week"] <= week + WEEK_HIST_MAX)]
    hist_df = hist_df.groupby("customer_id").agg({"article_id": list, "week": list}).reset_index()
    hist_df.rename(columns={"week": 'week_history'}, inplace=True)
    
    return test_df.merge(hist_df, on="customer_id", how="left")

test_df = create_test_dataset(test_df)
test_df.head()

In [ ]:
# 8割くらいの人がnull
# →80%の人はヒントなしになってしまっているので別の特徴量が必要そう
test_df["article_id"].isnull().mean()

# テストデータに対する推論 + submission.csvの作成

In [ ]:
test_ds = HMDataset(test_df, SEQ_LEN, is_test=True)
test_loader = DataLoader(test_ds, batch_size=BS, shuffle=False, num_workers=NW,
                          pin_memory=False, drop_last=False)


def inference(model, loader, k=12):
    model.eval()
    
    tbar = tqdm(loader, file=sys.stdout)
    
    preds = []
    
    with torch.no_grad():
        for idx, data in enumerate(tbar):
            inputs, target = read_data(data)

            logits = model(inputs)

            _, indices = torch.topk(logits, k, dim=1)

            indices = indices.detach().cpu().numpy()
            target = target.detach().cpu().numpy()

            for i in range(indices.shape[0]):
                preds.append(" ".join(list(le_article.inverse_transform(indices[i]))))
        
    
    return preds


test_df["prediction"] = inference(model, test_loader)

In [ ]:
test_df.to_csv("submission.csv", index=False, columns=["customer_id", "prediction"])

In [ ]:
# fin!